# Natural Language Inference
### Using an ensemble Architecture with Attention and Local Inference Modelling

This notebook runs the model on the test set.
Hyperparameters are set to the same values as they are in the training pipeline.



In [51]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf

In [52]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

Set the hyperparameters that were selected for training.

In [53]:
VOCAB_SIZE = 20000
MAX_LENGTH = 35

Reads the specificied csv file and splits into required columns.

In [54]:
def load_data(csv_path):
    """Load dataset from CSV file"""
    df = pd.read_csv(csv_path)
    return df["premise"].astype(str).tolist(), df["hypothesis"].astype(str).tolist()

In [55]:
test_data_path = input("Input the path to test data: ")
test_premises, test_hypotheses = load_data(test_data_path)

Input the path to test data:  /kaggle/input/nlu-test-dataset/test.csv


Load the tokeniser that was created during training.

In [57]:
import pickle

tokenizer = input("Input the path to tokenizer: ")
with open(tokenizer, "rb") as f:
    tokenizer = pickle.load(f)

Tokenise the input sequences and pad them to the maximum length.

In [58]:
def tokenize_and_pad(texts, tokenizer, max_length):
    """Convert text to sequences and pad"""
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

In [59]:
X_test_p = tokenize_and_pad(test_premises, tokenizer, MAX_LENGTH)
X_test_h = tokenize_and_pad(test_hypotheses, tokenizer, MAX_LENGTH)

Make sure to use the GloVe 6B, 300 dimensional embeddings file for the pretrained embeddings.

In [60]:
EMBEDDING_DIM = 300

In [61]:
def load_glove_embeddings(glove_file):
    """
    Load the embeddings for every word in the file
    """
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

In [62]:
glove_path = input("Input the path to glove embeddings file: ")
embeddings_index = load_glove_embeddings(glove_path, EMBEDDING_DIM)

Input the path to glove embeddings file:  /kaggle/input/glove300/glove.6B.300d.txt


Similar to training, create the embedding matrix for every word in the tokeniser.

In [63]:
def create_embedding_matrix(embedding_index, word_index, embedding_dim, vocab_size):
    """
    Creates an embedding matrix from the GloVe embeddings.
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    for word, index in word_index.items():
        if index >= vocab_size:
            break
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    
    return embedding_matrix

In [64]:
word_index = tokenizer.word_index
embedding_matrix = create_embedding_matrix(embeddings_index, word_index, EMBEDDING_DIM, VOCAB_SIZE)

Specify custom functions used in the model architecture.

In [66]:
def soft_attention(premise, hypothesis):
    attention = Dot(axes=-1)([premise, hypothesis])
    premise_attn = Softmax(axis=-1)(attention)
    hypothesis_attn = Softmax(axis=-2)(attention)

    premise_aligned = Dot(axes=1)([premise_attn, hypothesis])
    hypothesis_aligned = Dot(axes=1)([hypothesis_attn, premise])
    return premise_aligned, hypothesis_aligned

Specify paths to models.

In [67]:
lstm_path = input("Input the path to LSTM model: ")
bilstm_path = input("Input the path to BiLSTM model: ")
gru_path = input("Input the path to GRU model: ")
bigru_path = input("Input the path to BiGRU model: ")

Input the path to LSTM model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_LSTM_model.keras
Input the path to BiLSTM model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_BiLSTM_model.keras
Input the path to GRU model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_GRU_model.keras
Input the path to BiGRU model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_BiGRU_model.keras


In [68]:
from tensorflow.keras.models import load_model

In [69]:
models = {
    "LSTM": load_model(
        lstm_path,
        custom_objects={"soft_attention": soft_attention}
    ),
    "BiLSTM": load_model(
        bilstm_path,
        custom_objects={"soft_attention": soft_attention}
    ),
    "GRU": load_model(
        gru_path,
        custom_objects={"soft_attention": soft_attention}
    ),
    "BiGRU": load_model(
        bigru_path,
        custom_objects={"soft_attention": soft_attention}
    ),
}

Load the saved model weights for weighted averaging.

In [70]:
import json

weights_path = input("Input the path to model accuracy weights: ")
with open(weights_path, "r") as f:
    weights = json.load(f)

Input the path to model accuracy weights:  /kaggle/input/testing/model_accuracy_weights.json


Compute predictions.

In [71]:
weighted_sum = np.zeros((X_test_p.shape[0], 2), dtype=np.float32)
    
for name, model in models.items():
    raw_preds = model.predict([X_test_p, X_test_h])

    # Apply weighting and accumulate
    weighted_preds = raw_preds * weights[name]
    weighted_sum += weighted_preds

# Calculate final predictions
y_pred = np.argmax(weighted_sum, axis=1)

104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step


Save the predictions to a csv file with required name.

In [72]:
def save_predictions_only(y_pred, output_path):
    """Save only the predictions to a CSV file with a 'predictions' column."""
    df = pd.DataFrame({"predictions": y_pred.flatten()})
    df.to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}")

In [73]:
save_predictions_only(y_pred, "Group_47_B.csv")

Predictions saved to Group_47_B.csv
